# Static joining

In [8]:
import pandas as pd
import numpy as np

In [9]:
df1 = pd.read_csv(r"data\CAMELS_IND_Catchments_Streamflow_Sufficient\attributes_csv\camels_ind_anth.csv")
df2 = pd.read_csv(r"data\CAMELS_IND_Catchments_Streamflow_Sufficient\attributes_csv\camels_ind_clim.csv")
df3 = pd.read_csv(r"data\CAMELS_IND_Catchments_Streamflow_Sufficient\attributes_csv\camels_ind_geol.csv")
df4 = pd.read_csv(r"data\CAMELS_IND_Catchments_Streamflow_Sufficient\attributes_csv\camels_ind_hydro.csv")
df5 = pd.read_csv(r"data\CAMELS_IND_Catchments_Streamflow_Sufficient\attributes_csv\camels_ind_land.csv")
df6 = pd.read_csv(r"data\CAMELS_IND_Catchments_Streamflow_Sufficient\attributes_csv\camels_ind_name.csv")
df7 = pd.read_csv(r"data\CAMELS_IND_Catchments_Streamflow_Sufficient\attributes_csv\camels_ind_soil.csv")
df8 = pd.read_csv(r"data\CAMELS_IND_Catchments_Streamflow_Sufficient\attributes_csv\camels_ind_topo.csv")

In [10]:
# Merge all dataframes on gauge_id to include all gauge IDs
df_merged = df1.merge(df2, on='gauge_id', how='outer') \
              .merge(df3, on='gauge_id', how='outer') \
              .merge(df4, on='gauge_id', how='outer') \
              .merge(df5, on='gauge_id', how='outer') \
              .merge(df6, on='gauge_id', how='outer') \
              .merge(df7, on='gauge_id', how='outer') \
              .merge(df8, on='gauge_id', how='outer')

In [11]:
# Check the shape and number of unique gauge IDs
print(f"Merged DataFrame shape: {df_merged.shape}")
print(f"Number of unique gauge IDs: {df_merged['gauge_id'].nunique()}")
print(f"Number of columns: {len(df_merged.columns)}")

Merged DataFrame shape: (242, 211)
Number of unique gauge IDs: 242
Number of columns: 211


In [12]:
# Check for any duplicate columns (shouldn't be any with proper merge)
duplicate_cols = df_merged.columns[df_merged.columns.duplicated()].tolist()
print(f"Duplicate columns: {duplicate_cols}")

Duplicate columns: []


In [13]:
# Remove any duplicate columns if they exist
df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]

In [14]:
# Save the complete merged dataset
df_merged.to_csv("static_all_gauges.csv", index=False)
print("Merged dataset saved as 'static_all_gauges.csv'")

Merged dataset saved as 'static_all_gauges.csv'


# Stactic Data Dimn Redn

In [1]:
import pandas as pd
import numpy as np

In [7]:
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
import warnings
warnings.filterwarnings('ignore')

print("=== UMAP DIMENSIONALITY REDUCTION FOR STATIC DATA ===\n")

df_static = pd.read_csv("gauge_data/static_all_gauges.csv")
print(f"Original static dataset shape: {df_static.shape}")

numerical_cols = df_static.select_dtypes(include=[np.number]).columns.tolist()
print(f"Number of numerical columns: {len(numerical_cols)}")

if 'gauge_id' in numerical_cols:
    numerical_cols.remove('gauge_id')

df_numeric = df_static[numerical_cols].copy()

categorical_cols = df_static.select_dtypes(include=['object', 'string']).columns.tolist()
print(f"Number of categorical columns: {len(categorical_cols)}")

# Encode categorical variables if they exist
if categorical_cols:
    print("Encoding categorical variables...")
    df_categorical_encoded = df_static[categorical_cols].copy()
    
    # Use OrdinalEncoder for categorical variables
    encoders = {}
    for col in categorical_cols:
        if col != 'gauge_id':  # Skip gauge_id
            encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
            df_categorical_encoded[col] = encoder.fit_transform(df_categorical_encoded[[col]])
            encoders[col] = encoder
    
    # Combine numerical and encoded categorical data
    df_for_umap = pd.concat([df_numeric, df_categorical_encoded], axis=1)
else:
    df_for_umap = df_numeric.copy()

print(f"Data for UMAP shape: {df_for_umap.shape}")

# Handle missing values
print(f"Missing values before cleaning: {df_for_umap.isnull().sum().sum()}")
df_for_umap = df_for_umap.fillna(df_for_umap.median())
print(f"Missing values after cleaning: {df_for_umap.isnull().sum().sum()}")

# Standardize the features
print("Standardizing features...")
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_for_umap)
print(f"Scaled dataset shape: {df_scaled.shape}")


=== UMAP DIMENSIONALITY REDUCTION FOR STATIC DATA ===

Original static dataset shape: (242, 211)
Number of numerical columns: 198
Number of categorical columns: 13
Encoding categorical variables...
Data for UMAP shape: (242, 210)
Missing values before cleaning: 3463
Missing values after cleaning: 0
Standardizing features...
Scaled dataset shape: (242, 210)


In [12]:
n_dim = 40

# Create UMAP with current dimension
umap_model = umap.UMAP(
    n_components=n_dim, 
    random_state=42, 
    n_neighbors=15, 
    min_dist=0.1,
    n_epochs=1000,
    verbose=False
)

embedding_nd = umap_model.fit_transform(df_scaled)

# Create base dataframe with gauge_id for joining
base_df = df_static[['gauge_id']].copy()

column_names = [f"umap_{n_dim}d_dim_{i+1}" for i in range(n_dim)]
embedding_df = pd.DataFrame(embedding_nd, columns=column_names)

# Add gauge_id
embedding_with_id = pd.concat([base_df, embedding_df], axis=1)

embedding_with_id.to_csv(f"output/static_all_umap_{n_dim}d.csv", index=False)
print(f"Saved  UMAP embedding to 'static_umap_{n_dim}d.csv'")

Saved  UMAP embedding to 'static_umap_40d.csv'


# Static and Dynamic joining

In [16]:
import os
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

dynamic_dir = "data/CAMELS_IND_Catchments_Streamflow_Sufficient/catchment_mean_forcings"
output_dir = "output/dynamic"
os.makedirs(output_dir, exist_ok=True)

df_static = pd.read_csv("output/static_all_umap_40d.csv")
df_baseflow = pd.read_csv("gauge_data/dynamic/boughton_baseflow_all_stations.csv")

# Print available gauge_ids for debugging
print(f"Available gauge_ids in static data: {sorted(df_static['gauge_id'].unique())}")
print(f"Total static gauge_ids: {len(df_static['gauge_id'].unique())}")
skipped_gauge_ids = []
merged_gauge_ids = []

for fname in os.listdir(dynamic_dir):
    # Only process files with integer names (e.g., 3002.csv)
    if fname.endswith(".csv") and fname.split(".")[0].isdigit():
        gauge_id_str = fname.split(".")[0] 
        gauge_id = int(gauge_id_str)
        
        # Check if gauge_id exists in static data
        static_match = df_static[df_static['gauge_id'] == gauge_id]
        if static_match.empty:
            print(f"Skipping gauge_id {gauge_id}: not found in static data")
            skipped_gauge_ids.append(gauge_id)
            continue
            
        print(f"Processing gauge_id: {gauge_id} from file: {fname}")
        df_dyn = pd.read_csv(os.path.join(dynamic_dir, fname))
        
        if gauge_id_str in df_baseflow.columns:
            df_dyn = pd.concat([df_dyn, df_baseflow[gauge_id_str]], axis=1)
            df_dyn = df_dyn.rename(columns={df_dyn.columns[-1]: 'baseflow'})
            
        if 'date' in df_dyn.columns:
            df_dyn = df_dyn.drop('date', axis=1)
            
        row = static_match.reset_index(drop=True).iloc[0]
        n = len(df_dyn)
        duplicates = pd.DataFrame([row] * n, index=df_dyn.index)
        merged_df = pd.concat([df_dyn, duplicates], axis=1)

        if 'gauge_id' in merged_df.columns:
            merged_df = merged_df.drop(columns=['gauge_id'])

        cat = merged_df.select_dtypes(include=['object', 'string']).columns

        for column in cat:
            le = OrdinalEncoder()
            reshaped = merged_df[[column]]
            merged_df[column] = le.fit_transform(reshaped)
            
        merged_df.to_csv(os.path.join(output_dir, f"merged_df_{gauge_id}.csv"), index=False)
        print(f"Saved merged data for gauge_id {gauge_id}")
        merged_gauge_ids.append(gauge_id)

print("Processing completed!")

Available gauge_ids in static data: [np.int64(3002), np.int64(3005), np.int64(3008), np.int64(3009), np.int64(3013), np.int64(3014), np.int64(3017), np.int64(3018), np.int64(3019), np.int64(3020), np.int64(3024), np.int64(3028), np.int64(3029), np.int64(3030), np.int64(3031), np.int64(3038), np.int64(3040), np.int64(3043), np.int64(3045), np.int64(3047), np.int64(3048), np.int64(3055), np.int64(3056), np.int64(3057), np.int64(3064), np.int64(3068), np.int64(3069), np.int64(3071), np.int64(3072), np.int64(3073), np.int64(3074), np.int64(3075), np.int64(3076), np.int64(3079), np.int64(3081), np.int64(3082), np.int64(3083), np.int64(3085), np.int64(3087), np.int64(3089), np.int64(3091), np.int64(3092), np.int64(3094), np.int64(3098), np.int64(3099), np.int64(3103), np.int64(3105), np.int64(3107), np.int64(3109), np.int64(3110), np.int64(4003), np.int64(4004), np.int64(4005), np.int64(4006), np.int64(4007), np.int64(4012), np.int64(4013), np.int64(4015), np.int64(4016), np.int64(4017), np.

In [17]:
skipped_gauge_ids = sorted(skipped_gauge_ids)
print(f"Skipped gauge_ids (not found in static data): {skipped_gauge_ids}")

Skipped gauge_ids (not found in static data): [3042, 6009, 12002, 12005, 12007, 12010, 12013, 12017, 12019, 12027, 12030, 12037, 12039, 15002, 16015, 17002]


In [18]:
len_skipped = len(skipped_gauge_ids)
print(f"Total skipped gauge_ids: {len_skipped}")

Total skipped gauge_ids: 16


In [21]:
merged_gauge_ids = sorted(merged_gauge_ids)
print(f"Total merged gauge_ids: {(merged_gauge_ids)}")

Total merged gauge_ids: [3002, 3013, 3017, 3024, 3038, 3045, 3047, 3071, 3081, 3082, 3083, 3092, 3099, 3105, 3107, 4005, 4012, 4013, 4016, 4018, 4022, 4025, 4028, 4032, 4039, 4041, 4047, 4048, 4050, 4055, 4056, 4060, 5004, 5005, 5007, 5010, 5013, 5017, 5022, 5023, 5024, 5027, 5028, 6007, 6008, 7001, 7002, 7005, 7007, 7008, 7009, 7015, 8001, 8005, 8006, 8009, 8011, 8013, 8016, 8020, 8021, 8023, 8028, 8030, 8031, 8034, 8035, 8036, 8038, 9001, 9005, 9008, 10004, 10010, 10013, 11005, 11006, 11007, 12016, 12026, 12043, 12044, 12046, 13004, 13007, 13010, 13013, 13016, 14003, 14008, 14012, 14014, 15001, 15006, 15007, 15008, 15009, 15010, 15012, 15013, 15014, 15016, 15017, 15019, 15022, 15023, 15024, 15026, 15030, 15032, 16001, 16004, 16012, 16014, 17001, 17003, 17005, 17006, 17009, 17012, 17019]
